# Creating Coastal Watershed

This script processes spatial data to delineate watersheds specifically for coastal wetlands within the Great Lakes region. The workflow involves leveraging shapefiles provided by the Michigan Tech Research Institute (MTRI) and hydrological data to achieve this goal. The primary inputs include:

- **Coastal wetlands shapefiles**: Represent key inundation zones.
- **Great Lakes basin streams**.
- **D8 flow direction data**.

### Overview of the Workflow:

1. **Stream Selection**: 
   - Identify and extract streams that drain directly into the coastal wetlands.
   - Ensure only relevant hydrological pathways are considered for watershed delineation.

2. **Watershed Creation**:
   - Use coastal wetlands and D8 flow direction data to delineate watersheds feeding into these wetlands.

3. **Overlap Removal**:
   - Adjust resulting coastal watersheds by removing areas overlapping with stream watersheds.
   - Refine delineations to reflect distinct watershed boundaries.


# Import Libraries

In [1]:
# imports
### Reqiured Imports###
import pandas, os
import arcpy
from matplotlib import pyplot as plt

# Inputs

In [5]:
inDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer"
# define the path to coastal wetlands shapefile
inDCW = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\shapefiles"

# Coastal Wetlands shapefiles with different inundation levels
wetlands_avg_inun = os.path.join(inDCW, "Wetland_connected_avg_inundation_NAD1983")
wetlands_high_inun = os.path.join(inDCW, "wetlands_connected_high_inundation_NAD1983")
wetlands_low_inun = os.path.join(inDCW, "wetlands_connected_low_inundation_NAD1983")
wetlands_surge = os.path.join(inDCW, "wetlands_connected_surge_inundation_NAD1983")

# define the path to streams 
inStreams = os.path.join(inDir, "GLB_Stream", "GLB_stream_Ras_FeatureToLine.shp")

# define path to flow direction raster
D8_flow  = r"S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_dir.tif"

#path to stream watershed

inStreamsWatershed = os.path.join(inDir, "Streamwatershed", "PointWaterdhed_LH.shp")

# path to Lake Huron shapefile
Lake_Huron = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\hydro_p_LakeHuron\hydro_p_LakeHuron_84.shp"

# Outputs

In [21]:
# Buffer inStreams by 50 meters 
outDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastaWatersheds\GLB_Stream"

outBuffer = os.path.join(outDir, "GLB_stream_Ras_FeatureToLine_50m.shp")

#outBuffer = r"D:\Users\abolmaal\code\CoastalWatersheds\GLB_Stream\Buffer_50m\GLB_stream_Ras_FeatureToLine_Buffer_50m.shp"

outpath = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\CoastaWatersheds"

# path to Riperian shapefile
outErase_Riper = os.path.join(outpath,"Erase_Riperian")

erase_buffer_avg = os.path.join(outpath,"Erase_Riperian", "Wetland_connected_avg_erasebuff_50.shp")
erase_buffer_high = os.path.join(outpath,"Erase_Riperian", "Wetland_connected_high_erasebuff_50.shp")
erase_buffer_low = os.path.join(outpath,"Erase_Riperian", "Wetland_connected_low_erasebuff_50.shp")
erase_buffer_surge = os.path.join(outpath,"Erase_Riperian", "Wetland_connected_surge_erasebuff_50.shp")

# path to coastal Watersheds shapefile
# Stream Watershed raster outputs
erase_buffer_avg_watershed = os.path.join(outErase_Riper, "Wetland_connected_avg_watershed.tif")
erase_buffer_high_watershed = os.path.join(outErase_Riper, "Wetland_connected_high_watershed.tif")
erase_buffer_low_watershed = os.path.join(outErase_Riper, "Wetland_connected_low_watershed.tif")
erase_buffer_surge_watershed = os.path.join(outErase_Riper, "Wetland_connected_surge_watershed.tif")

#Stream Watershed outputs shapefile
erase_buffer_avg_watershed_poly = os.path.join(outErase_Riper, "Wetland_connected_avg_watershed.shp")
erase_buffer_high_watershed_poly = os.path.join(outErase_Riper, "Wetland_connected_high_watershed.shp")
erase_buffer_low_watershed_poly = os.path.join(outErase_Riper, "Wetland_connected_low_watershed.shp")
erase_buffer_surge_watershed_poly = os.path.join(outErase_Riper, "Wetland_connected_surge_watershed.shp")

# path to drainage shapefile
outErase_drainage = os.path.join (outpath,"Erase_drainage")
CoastalWatershed_avg_erase_lakedrain = os.path.join(outErase_drainage, "CoastalWatershed_avg_erase_lakedrain.shp")
CoastaWatershed_high_erase_lakedrain = os.path.join(outErase_drainage, "CoastalWatershed_high_erase_lakedrain.shp")
CoastaWatershed_low_erase_lakedrain = os.path.join(outErase_drainage, "CoastalWatershed_low_erase_lakedrain.shp")
CoastaWatershed_surge_erase_lakedrain = os.path.join(outErase_drainage, "CoastalWatershed_surge_erase_lakedrain.shp")


# path to Lake Huron shapefile
outErase_Lake = os.path.join (outpath,"Erase_lake")
# path to erase lake shapefile from coastal watershed
CoastalWatershed_avg_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake, "CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp")
CoastalWatershed_high_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake, "CoastalWatershed_high_erase_lakedrain_LakeHuron.shp")
CoastalWatershed_low_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake, "CoastalWatershed_low_erase_lakedrain_LakeHuron.shp")
CoastalWatershed_surge_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake, "CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp")

# out name for stream watershed set to WGS84 coordinate 
CoastalWatershed_avg_erase_lakedrain_LakeHuron_WGS1984 = os.path.join(outErase_Lake, "CoastalWatershed_avg_erase_lakedrain_LakeHuron_WGS1984.shp")
CoastalWatershed_high_erase_lakedrain_LakeHuron_WGS1984 = os.path.join(outErase_Lake, "CoastalWatershed_high_erase_lakedrain_LakeHuron_WGS1984.shp")
CoastalWatershed_low_erase_lakedrain_LakeHuron_WGS1984 = os.path.join(outErase_Lake, "CoastalWatershed_low_erase_lakedrain_LakeHuron_WGS1984.shp")
CoastalWatershed_surge_erase_lakedrain_LakeHuron_WGS1984 = os.path.join(outErase_Lake, "CoastalWatershed_surge_erase_lakedrain_LakeHuron_WGS1984.shp")


# Main Functions

## 1- Buffer the streams
Create a 50 meter buffer for Great lakes basin streams to create your riverin riperian area

In [6]:
# buffer inStreams by 50 meters
arcpy.Buffer_analysis(inStreams, outBuffer, "50 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\GLB_Stream\\GLB_stream_Ras_FeatureToLine_50m.shp'>

## 2- Erase your coastal wetlands that overlap with your riperian area

In [7]:
# Erase the buffer from the wetlands shapefile
# erase_analisys (input, erase_feature, output_feature, {cluster_tolerance})

# Erase the outBuffer from the wetlands_avg_inun shapefile with the output name erase_buffer_avg
arcpy.Erase_analysis(wetlands_avg_inun, outBuffer, erase_buffer_avg, None)

# Erase the outBuffer from the wetlands_high_inun shapefile with the output name erase_buffer_high
arcpy.Erase_analysis(wetlands_high_inun, outBuffer, erase_buffer_high, None)

# Erase the outBuffer from the wetlands_low_inun shapefile with the output name erase_buffer_low
arcpy.Erase_analysis(wetlands_low_inun, outBuffer, erase_buffer_low, None)

# Erase the outBuffer from the wetlands_surge shapefile with the output name erase_buffer_surge
arcpy.Erase_analysis(wetlands_surge, outBuffer, erase_buffer_surge, None)

<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_Riperian\\Wetland_connected_surge_erasebuff_50.shp'>

#### 2-1 Add a field called coastal_Id

In [8]:
# add a field called coastal_id to the erase_buffer_avg shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_avg, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_avg, "coastal_id", "!FID! + 1", "PYTHON3")

# add a field called coastal_id to the erase_buffer_high shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_high, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_high, "coastal_id", "!FID! + 1", "PYTHON3")

# add a field called coastal_id to the erase_buffer_low shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_low, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_low, "coastal_id", "!FID! + 1", "PYTHON3")

# add a field called coastal_id to the erase_buffer_surge shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_surge, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_surge, "coastal_id", "!FID! + 1", "PYTHON3")


<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_Riperian\\Wetland_connected_surge_erasebuff_50.shp'>

## 3- Create a watershed

### 3-1 Convert the polygons to raster
Watershed tool only accept raster as input so here we first
convert erase_buffer_avg polygon to raster with the output name erase_buffer_avg_raster

In [9]:
# convert erase_buffer_avg polygon to raster with the output name erase_buffer_avg_raster
erase_buffer_avg_raster = os.path.join(outErase_Riper, "Wetland_connected_avg_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_avg, "coastal_Id", erase_buffer_avg_raster, "CELL_CENTER", "NONE", "50")

# convert erase_buffer_high polygon to raster with the output name erase_buffer_high_raster
erase_buffer_high_raster = os.path.join(outErase_Riper, "Wetland_connected_high_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_high, "coastal_Id", erase_buffer_high_raster, "CELL_CENTER", "NONE", "50")

# convert erase_buffer_low polygon to raster with the output name erase_buffer_low_raster
erase_buffer_low_raster = os.path.join(outErase_Riper, "Wetland_connected_low_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_low, "coastal_Id", erase_buffer_low_raster, "CELL_CENTER", "NONE", "50")

## convert erase_buffer_surge polygon to raster with the output name erase_buffer_low_raster
erase_buffer_surge_raster = os.path.join(outErase_Riper, "Wetland_connected_surge_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_surge, "coastal_Id", erase_buffer_surge_raster, "CELL_CENTER", "NONE", "50")



<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_Riperian\\Wetland_connected_surge_raster.tif'>

### 3-2 Create Watershed
example arcpy.gp.watershed(flowdirection, coatalwetlands, outputpath)

In [11]:

# Create watershed for erase_buffer_avg_raster with the output name erase_buffer_avg_watershed 

arcpy.gp.Watershed_sa(D8_flow, erase_buffer_avg_raster, erase_buffer_avg_watershed)

# Create watershed for erase_buffer_high_raster with the output name erase_buffer_high_watershed 

arcpy.gp.Watershed_sa(D8_flow, erase_buffer_high_raster, erase_buffer_high_watershed)

# Create watershed for erase_buffer_low_raster with the output name erase_buffer_low_watershed 

arcpy.gp.Watershed_sa(D8_flow, erase_buffer_low_raster, erase_buffer_low_watershed)

# Create watershed for erase_buffer_surge_raster with the output name erase_buffer_surge_watershed 

arcpy.gp.Watershed_sa(D8_flow, erase_buffer_surge_raster, erase_buffer_surge_watershed)



<geoprocessing server result object at 0x1ceee6580f0>

### 3-3 convert the raster watershed to shapefiles 

In [13]:
# convert erase_buffer_avg_watershed to polygon with the output name erase_buffer_avg_watershed_poly

arcpy.RasterToPolygon_conversion(erase_buffer_avg_watershed, erase_buffer_avg_watershed_poly, "SIMPLIFY", "VALUE")

# convert erase_buffer_high_watershed to polygon with the output name erase_buffer_high_watershed_poly

arcpy.RasterToPolygon_conversion(erase_buffer_high_watershed, erase_buffer_high_watershed_poly, "SIMPLIFY", "VALUE")

# convert erase_buffer_low_watershed to polygon with the output name erase_buffer_low_watershed_poly

arcpy.RasterToPolygon_conversion(erase_buffer_low_watershed, erase_buffer_low_watershed_poly, "SIMPLIFY", "VALUE")

# convert erase_buffer_surge_watershed to polygon with the output name erase_buffer_surge_watershed_poly

arcpy.RasterToPolygon_conversion(erase_buffer_surge_watershed, erase_buffer_surge_watershed_poly, "SIMPLIFY", "VALUE")

<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_Riperian\\Wetland_connected_surge_watershed.shp'>

## 4- Erase Lake drainage watersed from coastal watershed
In this part we are removing the part of coastal watersheds that overlaps with stream watershed

In [15]:
# Erase the erase_buffer_avg_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_avg_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID+1


arcpy.Erase_analysis(erase_buffer_avg_watershed_poly,inStreamsWatershed, CoastalWatershed_avg_erase_lakedrain, None)
arcpy.CalculateField_management(CoastalWatershed_avg_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")

# Erase the erase_buffer_high_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_high_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID +1

arcpy.Erase_analysis(erase_buffer_high_watershed_poly, inStreamsWatershed, CoastaWatershed_high_erase_lakedrain, None)
arcpy.CalculateField_management(CoastaWatershed_high_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")


# Erase the erase_buffer_low_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_low_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID +1

arcpy.Erase_analysis(erase_buffer_low_watershed_poly,inStreamsWatershed, CoastaWatershed_low_erase_lakedrain, None)
arcpy.CalculateField_management(CoastaWatershed_low_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")

# Erase the erase_buffer_surge_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_surge_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID +1


arcpy.Erase_analysis(erase_buffer_surge_watershed_poly, inStreamsWatershed, CoastaWatershed_surge_erase_lakedrain, None)
arcpy.CalculateField_management(CoastaWatershed_surge_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")



<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_drainage\\CoastalWatershed_surge_erase_lakedrain.shp'>

### 4-1 Calculate shape area 

In [16]:
# Add a field called Shape_Area to the CoastalWatershed_avg_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastalWatershed_avg_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_avg_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")

# Add a field called Shape_Area to the CoastaWatershed_high_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastaWatershed_high_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastaWatershed_high_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")

# Add a field called Shape_Area to the CoastaWatershed_low_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastaWatershed_low_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastaWatershed_low_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")

# Add a field called Shape_Area to the CoastaWatershed_surge_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastaWatershed_surge_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastaWatershed_surge_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")


<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_drainage\\CoastalWatershed_surge_erase_lakedrain.shp'>

### 4-2 remove the parts of watershed inside the lake 

Some parts of coastal watershed are in the lake, we need to Earse the parts that are inside the Lake

In [19]:
# Erase the CoastalWatershed_avg_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp 
# and save it in outErase_Lake_path

arcpy.Erase_analysis(CoastalWatershed_avg_erase_lakedrain, Lake_Huron, CoastalWatershed_avg_erase_lakedrain_LakeHuron, None)


# Erase the CoastaWatershed_high_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_high_erase_lakedrain_LakeHuron.shp
# and save it in outErase_Lake_path

arcpy.Erase_analysis(CoastaWatershed_high_erase_lakedrain, Lake_Huron, CoastalWatershed_high_erase_lakedrain_LakeHuron, None)

# Erase the CoastaWatershed_low_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_low_erase_lakedrain_LakeHuron.shp
# and save it in outErase_Lake_path

arcpy.Erase_analysis(CoastaWatershed_low_erase_lakedrain, Lake_Huron, CoastalWatershed_low_erase_lakedrain_LakeHuron, None)

# Erase the CoastaWatershed_surge_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp
# and save it in outErase_Lake_path

arcpy.Erase_analysis(CoastaWatershed_surge_erase_lakedrain, Lake_Huron, CoastalWatershed_surge_erase_lakedrain_LakeHuron, None)


<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_lake\\CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp'>

### 4-2 Add Latitude and Longitude field to coastal watershed

In [20]:
# add start_lat and start_lon fields to the CoastalWatershed_avg_erase_lakedrain_LakeHuron shapefile and calculate the geometry of each feature
arcpy.AddField_management(CoastalWatershed_avg_erase_lakedrain_LakeHuron, "start_lat", "DOUBLE")
arcpy.AddField_management(CoastalWatershed_avg_erase_lakedrain_LakeHuron, "start_lon", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_avg_erase_lakedrain_LakeHuron, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")

# add start_lat and start_lon fields to the CoastalWatershed_high_erase_lakedrain_LakeHuron shapefile and calculate the geometry of each feature
arcpy.AddField_management(CoastalWatershed_high_erase_lakedrain_LakeHuron, "start_lat", "DOUBLE")
arcpy.AddField_management(CoastalWatershed_high_erase_lakedrain_LakeHuron, "start_lon", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_high_erase_lakedrain_LakeHuron, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")

# add start_lat and start_lon fields to the CoastalWatershed_low_erase_lakedrain_LakeHuron shapefile and calculate the geometry of each feature
arcpy.AddField_management(CoastalWatershed_low_erase_lakedrain_LakeHuron, "start_lat", "DOUBLE")
arcpy.AddField_management(CoastalWatershed_low_erase_lakedrain_LakeHuron, "start_lon", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_low_erase_lakedrain_LakeHuron, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")

# add start_lat and start_lon fields to the CoastalWatershed_surge_erase_lakedrain_LakeHuron shapefile and calculate the geometry of each feature
arcpy.AddField_management(CoastalWatershed_surge_erase_lakedrain_LakeHuron, "start_lat", "DOUBLE")
arcpy.AddField_management(CoastalWatershed_surge_erase_lakedrain_LakeHuron, "start_lon", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_surge_erase_lakedrain_LakeHuron, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")


<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_lake\\CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp'>

### 4-3 change the coordinate 

In [22]:
# change the crs of the CoastalWatershed_avg_erase_lakedrain_LakeHuron shapefile to WGS 1984
arcpy.Project_management(CoastalWatershed_avg_erase_lakedrain_LakeHuron, CoastalWatershed_avg_erase_lakedrain_LakeHuron_WGS1984, arcpy.SpatialReference(4326))
# calculate start_lat and start_lon fields for the CoastalWatershed_avg_erase_lakedrain_LakeHuron_WGS1984 shapefile
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_avg_erase_lakedrain_LakeHuron_WGS1984, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")


# change the crs of the CoastalWatershed_high_erase_lakedrain_LakeHuron shapefile to WGS 1984
arcpy.Project_management(CoastalWatershed_high_erase_lakedrain_LakeHuron, CoastalWatershed_high_erase_lakedrain_LakeHuron_WGS1984, arcpy.SpatialReference(4326))
# calculate start_lat and start_lon fields for the CoastalWatershed_high_erase_lakedrain_LakeHuron_WGS1984 shapefile
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_high_erase_lakedrain_LakeHuron_WGS1984, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")

# change the crs of the CoastalWatershed_low_erase_lakedrain_LakeHuron shapefile to WGS 1984
arcpy.Project_management(CoastalWatershed_low_erase_lakedrain_LakeHuron, CoastalWatershed_low_erase_lakedrain_LakeHuron_WGS1984, arcpy.SpatialReference(4326))
# calculate start_lat and start_lon fields for the CoastalWatershed_low_erase_lakedrain_LakeHuron_WGS1984 shapefile
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_low_erase_lakedrain_LakeHuron_WGS1984, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")


# change the crs of the CoastalWatershed_surge_erase_lakedrain_LakeHuron shapefile to WGS 1984
arcpy.Project_management(CoastalWatershed_surge_erase_lakedrain_LakeHuron, CoastalWatershed_surge_erase_lakedrain_LakeHuron_WGS1984, arcpy.SpatialReference(4326))
# calculate start_lat and start_lon fields for the CoastalWatershed_surge_erase_lakedrain_LakeHuron_WGS1984 shapefile
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_surge_erase_lakedrain_LakeHuron_WGS1984, [["start_lat", "CENTROID_Y"], ["start_lon", "CENTROID_X"]], area_unit="SQUARE_METERS")



<Result 'D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\CoastaWatersheds\\Erase_lake\\CoastalWatershed_surge_erase_lakedrain_LakeHuron_WGS1984.shp'>